# Prepare pdfs for later in pipeline (Obj Det, Img, text, NER)

- user provides
    - Google Cloud project (input)
    - bucket in GCS of pdfs (input)
    - BQ dataset to write prediction results (output)
        - BQ table: aggregated results (pdf_name, icn_pred, objdet_pred(coords), text_cn, ner1, ner2, ...., ner)
            created with JOIN on pdf_name
        - BQ table: icn_preds (pdf_name, icn_pred)    --> this table is made in icn_predict.ipynb
        - BQ table: objdet_pred (pdf_name, objdet_pred(coords)) --> this table is made in objdet_predict.ipynb
        - BQ table: text_cn (pdf_name, text_cn)    --> this table is made in text_cn_predict.ipynb
        - BQ table: ner (pdf_name, ner1, ner2, ...., ner)
        
- see utils.py for utils functions
        

Steps: 
 1. convert pdf to png and write to bucket (for ICN, ObjDet)
 2. do ocr on pdf and write to bucket 
 3. create dataset 
    

In [ ]:
PROJECT = !gcloud config get-value project # returns SList
PROJECT = PROJECT[0] # gets first element in list -> str
REGION = "us-central1"  
MODEL_RESOURCE_NAME = "2393478483993952256"

os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION

In [ ]:
import os
from google.cloud import bigquery
bq = bigquery.Client(project=PROJECT)

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
from google.cloud import storage
import tempfile



In [ ]:
class Utils():
    def __init__(self):
        self.storage_client = storage.Client()
    
    def convert_pdf_to_png(self, src_bucket_name, dst_bucket_name):
        """Takes pdfs from src_bucket_name and transforms them into png. Then it saves the result in dst_bucket_name"""
        try:
            src_bucket = self.storage_client.bucket(src_bucket_name)
            dst_bucket = self.storage_client.bucket(dst_bucket_name)

            blob_list = [blob for blob in list(bucket.list_blobs()) if \
                         os.path.basename(src_bucket_name) in blob.name and \
                         blob.name.endswith(".pdf")]

            encoded_img_lst = []

            # step 1: create images
            for blob in blob_list:
                _, tmp_pdf = tempfile.mkstemp()
                blob.download_to_filename(tmp_pdf)

                image = convert_from_path(tmp_pdf)
                image = image[0]                # Only the firs page is going to be analyzed.
                img_byte_arr = io.BytesIO()
                image.save(img_byte_arr, format='PNG')
                img_byte_arr = img_byte_arr.getvalue()
                encoded_img_lst.append((blob.name, base64.b64encode(img_byte_arr).decode("utf-8")))


            "gs://bucket/folder/example.pdf"
            "gs://bucket/example.pdf"

            # step 2: save images in dst_bucket_name
            for name, encoded_img in encoded_img_list:
                blob = dst_bucket.blob(name+".png")
                # TODO: Test please
                blob.upload_from_string(encoded_img.tobytes())
                
            return True
        
        except Exception as e:
            logging.error(f"some error occured: {e}")
            return False
    
    def ocr(self, src_bucket_name, dst_bucket_name):
        pass
    
    def convert_results_to_df(self, results):
        pass
    
    def upload_csv_to_storage(self, csv_path, gcs_path):
        pass
    
    def upload_csv_to_bigquery(self, csv_path, dataset_id, table_id, schema):
        pass